# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to Moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model
**Q1**: Expand the above definition of $ p(\vec{w})$ using naive estimates of the parameters, such as $  p(w_4 \mid w_2, w_3) {=}  \frac{C(w_2~w_3~w_4)}{C(w_2~w_3)} $ where \( C(w_2 w_3 w_4) \) denotes the count of times the trigram $ w_2 w_3 w_4 $ was observed in a training corpus.

**Write your answer:**

$ p(\vec{w})$ =  $ p(w_1) ⋅ p(w_2 \mid w_1) ⋅ p(w_3 \mid w_1, w_2) ⋅(w_4 \mid w_2, w_3) ... p(w_n \mid w_{n-2}, w_{n-1})$ \\
$ = \dfrac{C(w_1)}{C(*)} \dfrac{C(w_1~w_2)}{C(w_1)} \dfrac{C(w_1~w_2~w_3)}{C(w_1~w_2)} \dfrac{C(w_2~w_3~w_4)}{C(w_2~w_3)} ... \dfrac{C(w_{n-2} ~w_{n-1} ~w_n)}{C(w_{n-2}~w_{n-1})}$




**Q2**: One could also define a kind of reversed trigram language model $p_{reversed}$ that instead assumed the words were generated in reverse order (from right to left):
\begin{align} p_{reversed}(\vec{w}) \stackrel{\tiny{\mbox{def}}}{=}&p(w_n) \cdot p(w_{n-1} \mid w_n) \cdot p(w_{n-2} \mid w_{n-1} w_n) \cdot p(w_{n-3} \mid w_{n-2} w_{n-1}) \\ &\cdots p(w_2 \mid w_3 w_4) \cdot p(w_1 \mid w_2 w_3) \end{align}
By manipulating the notation, show that the two models are identical, i.e., $ p(\vec{w}) = p_{reversed}(\vec{w}) $ for any $ \vec{w} $ provided that both models use MLE parameters estimated from the same training data (see Q1 above).

**Write your answer:**

The MLE of $ p(\vec{w})$ is $ C(w_1) \dfrac{C(w_1~w_2)}{C(w_1)} \dfrac{C(w_1~w_2~w_3)}{C(w_1~w_2)} \dfrac{C(w_2~w_3~w_4)}{C(w_2~w_3)} ... \dfrac{C(w_{n-2} ~w_{n-1} ~w_n)}{C(w_{n-2}~w_{n-1})}$ \\

Which can be canceled to: \\

$ C(w_1~w_2~w_3) \dfrac{C(w_2~w_3~w_4)}{C(w_2~w_3)} ... \dfrac{C(w_{n-2} ~w_{n-1} ~w_n)}{C(w_{n-2}~w_{n-1})}$ \\

Similarly, we can write the MLE of $ p_{reversed}(\vec{w}) $ to: $ C(w_n) \dfrac{C(w_{n-1}~w_n)}{C(w_n)} \dfrac{C(w_{n-2} ~w_{n-1} ~w_n)}{C(w_{n-1}~w_n)} \dfrac{C(w_{n-3}~w_{n-2}~w_{n-1})}{C(w_{n-2}~w_{n-1})} ... \dfrac{C(w_2~w_3~w_4)}{C(w_2~w_3)} \dfrac{C(w_1~w_2~w_3)}{C(w_1~w_2)} $ \\

Which can also be canceled to: \\

$ {C(w_{n-2} ~w_{n-1} ~w_n)} \dfrac{C(w_{n-3}~w_{n-2}~w_{n-1})}{C(w_{n-2}~w_{n-1})} ... \dfrac{C(w_2~w_3~w_4)}{C(w_2~w_3)} \dfrac{C(w_1~w_2~w_3)}{C(w_1~w_2)} $ \\

This is equavalent to the answer above, if we multiply all the denominators and numerators respectively, the composition are equavalent.




## 2 $N$-gram Language Model Implementation

In [ ]:
!wget -O train.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/test.txt

### 2.1 Building vocabulary

**Code**

\# todo



**Discussion**

\# todo



### 2.2 $N$-gram Language Modeling

**Code**

\# todo



**Discussion**

\# todo



### 2.3 Smoothing

#### 2.3.1 Add-one (Laplace) smoothing

**Code**

\# todo



**Discussion**

\# todo



#### 2.3.2: Add-$k$ smoothing

**Code**

\# todo



**Discussion**

\# todo



#### 2.3.3 Linear Interpolation

**Code**

\# todo



**Discussion**

\# todo



##### **Optimization**:

\# todo

## 3 Preposition Prediction

In [ ]:
!wget -O dev.in https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.in
!wget -O dev.out https://github.com/qtli/COMP7607-Fall2023/blob/master/assignments/A1/data/prep/dev.out